In [13]:
##ADD YOUR IMPORTS HERE...
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import os
import seaborn as sns
import re
import folium
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from jupyter_dash import JupyterDash

In [9]:
final_df = pd.read_csv('final_df.csv')

## TEAMS ON OUTCOMES

In [19]:
# Initialize the Dash app
app1 = dash.Dash(__name__)

# Layout of the app
app1.layout = html.Div([
    html.H1("NFL Analysis Based on Team"),
    
    dcc.Dropdown(
        id='visualization-dropdown',
        options=[
            {'label': 'Over/Under', 'value': 'Over/Under'},
            {'label': 'Favorite', 'value': 'Favorite'},
            {'label': 'Spread Coverage', 'value': 'Spread Coverage'}
        ],
        value='Over/Under',
        clearable=False
    ),
    
    dcc.Dropdown(
        id='team-dropdown',
        options=[
            {'label': 'Overall NFL', 'value': 'Overall NFL'}
        ] + [{'label': team, 'value': team}
             for team in sorted(pd.concat([final_df['team_home'], final_df['team_away']]).unique())],
        value='Overall NFL',
        clearable=False
    ),
    
    dcc.RadioItems(
        id='playoff-toggle',
        options=[
            {'label': 'Regular Season', 'value': 'False'},
            {'label': 'Playoffs', 'value': 'True'}
        ],
        value='False',
        labelStyle={'display': 'inline-block', 'marginRight': 20}
    ),
    
    dcc.Graph(id='bar-chart')
])

@app1.callback(
    Output('bar-chart', 'figure'),
    [Input('visualization-dropdown', 'value'),
     Input('team-dropdown', 'value'),
     Input('playoff-toggle', 'value')]
)
def update_graph1(selected_visualization, selected_team, is_playoff):
    # Filter the DataFrame based on playoff status
    filtered_df = final_df[final_df['schedule_playoff'].astype(str) == is_playoff]
    
    game_type = "Playoff" if is_playoff == 'True' else "Regular Season"
    
    def create_overall_chart(data_column, color_map, title):
        total_games = len(filtered_df)
        if total_games > 0:
            result_counts = filtered_df[data_column].value_counts()
            percentages = (result_counts / total_games * 100).to_dict()
            
            fig = go.Figure(data=[
                go.Bar(name=result, x=['Overall NFL'], y=[percentages[result]],
                       marker_color=color_map[result], 
                       text=f'{result}: {count}/{total_games} ({percentages[result]:.1f}%)')
                for result, count in result_counts.items()
            ])
            
            fig.update_layout(
                title=f'{game_type} {title} Distribution for Overall NFL',
                xaxis_title='Category',
                yaxis_title='Percentage',
                barmode='group',
                showlegend=True,
                height=400,
                margin=dict(l=100, r=50, t=50, b=50),
                xaxis=dict(
                    tickvals=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
                    ticktext=['0%', '10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%']
                )
            )
        else:
            fig = go.Figure()
            fig.update_layout(
                title=f'No Data Available for {game_type}',
                xaxis_title='Category',
                yaxis_title='Percentage',
                height=400,
                margin=dict(l=100, r=50, t=50, b=50)
            )
        return fig

    def create_team_chart(data_column, color_map, title):
        all_teams = pd.concat([filtered_df['team_home'], filtered_df['team_away']]).unique()
        all_teams = [team for team in sorted(all_teams) if team != selected_team]

        game_stats = []
        for opponent in all_teams:
            team_games = filtered_df[
                ((filtered_df['team_home'] == selected_team) & (filtered_df['team_away'] == opponent)) |
                ((filtered_df['team_home'] == opponent) & (filtered_df['team_away'] == selected_team))
            ]
            
            total_games = len(team_games)
            if total_games > 0:
                result_counts = team_games[data_column].value_counts()
                percentages = (result_counts / total_games * 100).to_dict()
                
                game_stats.append({
                    'team': opponent,
                    'total_games': total_games,
                    **{f'{result}_pct': percentages.get(result, 0) for result in color_map},
                    **{f'{result}_games': result_counts.get(result, 0) for result in color_map}
                })
        
        if not game_stats:
            fig = go.Figure()
            fig.update_layout(
                title=f'No Data Available for {game_type}',
                xaxis_title='Percentage',
                yaxis_title='Opponent Teams',
                height=400,
                margin=dict(l=100, r=50, t=50, b=50)
            )
        else:
            game_stats_sorted = sorted(game_stats, key=lambda x: x['total_games'], reverse=True)
            
            all_teams_sorted = [stats['team'] for stats in game_stats_sorted]
            
            fig = go.Figure()
            for result, color in color_map.items():
                percentages = [stats[f'{result}_pct'] for stats in game_stats_sorted]
                games = [stats[f'{result}_games'] for stats in game_stats_sorted]
                total_games = [stats['total_games'] for stats in game_stats_sorted]
                
                hover_text = [f"Team: {selected_team} vs {team}<br>{result}: {games}/{total} ({pct:.1f}%)<br>Total Games: {total}" 
                              for team, games, total, pct in zip(all_teams_sorted, games, total_games, percentages)]
                
                fig.add_trace(go.Bar(
                    name=result, y=all_teams_sorted, x=percentages,
                    marker_color=color, hoverinfo='text', hovertext=hover_text, orientation='h'
                ))
            
            annotations = []
            for i, (team, total) in enumerate(zip(all_teams_sorted, [stats['total_games'] for stats in game_stats_sorted])):
                annotations.append(dict(
                    x=1.02, y=team, xref='paper', yref='y',
                    text=str(total), showarrow=False,
                    font=dict(size=10, color='black'),
                    align='left', xanchor='left', yanchor='middle'
                ))
            
            annotations.append(dict(
                x=1.06, y=0.5, xref='paper', yref='paper',
                text='# of Games', showarrow=False,
                font=dict(size=12, color='black', weight='bold'),
                textangle=-90, align='center', xanchor='center', yanchor='middle'
            ))
            
            axis_title_style = dict(size=12, color='black', weight='bold')
            
            fig.update_layout(
                title=f'{game_type} {title} Distribution for {selected_team} vs All Other Teams',
                xaxis_title=dict(text='Percentage', font=axis_title_style),
                yaxis_title=dict(text='Opponent Teams', font=axis_title_style),
                barmode='relative',
                xaxis={
                    'range': [0, 100],
                    'tickmode': 'array',
                    'tickvals': list(range(0, 101, 10)),
                    'ticktext': [f'{i}%' for i in range(0, 101, 10)]
                },
                yaxis={'autorange': 'reversed'},
                showlegend=True,
                legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                hovermode='closest',
                height=800,
                margin=dict(l=200, r=100),
                shapes=[dict(
                    type='line', x0=50, x1=50, y0=0, y1=1,
                    xref='x', yref='paper',
                    line=dict(color='black', width=4)
                )],
                annotations=annotations,
                font=dict(family="Arial, sans-serif")
            )
        
        return fig

    if selected_visualization == 'Over/Under':
        color_map = {'Under': 'red', 'Over': 'green'}
        if selected_team == 'Overall NFL':
            return create_overall_chart('over_under_result', color_map, 'Over/Under')
        else:
            return create_team_chart('over_under_result', color_map, 'Over/Under')
    
    elif selected_visualization == 'Favorite':
        color_map = {False: 'red', True: 'green'}
        if selected_team == 'Overall NFL':
            return create_overall_chart('favorite_won', color_map, 'Favorite')
        else:
            return create_team_chart('favorite_won', color_map, 'Favorite')
    
    elif selected_visualization == 'Spread Coverage':
        color_map = {False: 'red', True: 'green'}
        if selected_team == 'Overall NFL':
            return create_overall_chart('favorite_covered', color_map, 'Spread Coverage')
        else:
            return create_team_chart('favorite_covered', color_map, 'Spread Coverage')

if __name__ == '__main__':
    app1.run_server(debug=True, port=8066)

## TYPES ON OUTCOMES

In [20]:
# Initialize 2nd Dash app
app2 = dash.Dash(__name__)

# Layout of the app
app2.layout = html.Div([
    html.H1("NFL Analysis Based on Location Factors"),
    
    dcc.Dropdown(
        id='visualization-dropdown',
        options=[
            {'label': 'Over/Under', 'value': 'Over/Under'},
            {'label': 'Favorite', 'value': 'Favorite'},
            {'label': 'Spread Coverage', 'value': 'Spread Coverage'}
        ],
        value='Over/Under',
        clearable=False
    ),
    
    dcc.Dropdown(
        id='type-dropdown',
        options=[
            {'label': 'Weather Type', 'value': 'Weather Type'},
            {'label': 'Stadium Type', 'value': 'Stadium Type'}
        ],
        value='Weather Type',
        clearable=False
    ),
    
    dcc.RadioItems(
        id='playoff-toggle',
        options=[
            {'label': 'Regular Season', 'value': 'False'},
            {'label': 'Playoffs', 'value': 'True'}
        ],
        value='False',
        labelStyle={'display': 'inline-block', 'marginRight': 20}
    ),
    
    dcc.Graph(id='bar-chart')
])

@app2.callback(
    Output('bar-chart', 'figure'),
    [Input('visualization-dropdown', 'value'),
     Input('type-dropdown', 'value'),
     Input('playoff-toggle', 'value')]
)
def update_graph2(selected_visualization, selected_type, is_playoff):
    # Filter the DataFrame based on playoff status
    filtered_df = final_df[final_df['schedule_playoff'].astype(str) == is_playoff]
    
    game_type = "Playoff" if is_playoff == 'True' else "Regular Season"
    
    # Determine which column to use for the dropdown
    column = 'stadium_weather_type' if selected_type == 'Weather Type' else 'stadium_type'
    
    fig = go.Figure()
    
    if selected_visualization == 'Over/Under':
        type_counts = filtered_df.groupby([column, 'over_under_result']).size().unstack(fill_value=0)
        type_percentages = (type_counts.T / type_counts.sum(axis=1)).T * 100
        
        if 'Under' in type_percentages.columns:
            fig.add_trace(go.Bar(
                y=type_percentages.index,
                x=type_percentages['Under'],
                name='Under',
                marker_color='red',
                orientation='h',
                textposition='none',
                hovertemplate=[
                    f'Under: {x:,}<br>Percentage: {perc:.2f}%<extra></extra>' 
                    for x, perc in zip(type_counts['Under'], type_percentages['Under'])
                ]
            ))
        if 'Over' in type_percentages.columns:
            fig.add_trace(go.Bar(
                y=type_percentages.index,
                x=type_percentages['Over'],
                name='Over',
                marker_color='green',
                orientation='h',
                textposition='none',
                hovertemplate=[
                    f'Over: {x:,}<br>Percentage: {perc:.2f}%<extra></extra>' 
                    for x, perc in zip(type_counts['Over'], type_percentages['Over'])
                ]
            ))

    elif selected_visualization == 'Favorite':
        type_counts = filtered_df.groupby([column, 'favorite_won']).size().unstack(fill_value=0)
        type_percentages = (type_counts.T / type_counts.sum(axis=1)).T * 100
        
        if False in type_percentages.columns:
            fig.add_trace(go.Bar(
                y=type_percentages.index,
                x=type_percentages[False],
                name='Favorite Lost',
                marker_color='red',
                orientation='h',
                textposition='none',
                hovertemplate=[
                    f'Favorite Lost: {x:,}<br>Percentage: {perc:.2f}%<extra></extra>' 
                    for x, perc in zip(type_counts[False], type_percentages[False])
                ]
            ))
        if True in type_percentages.columns:
            fig.add_trace(go.Bar(
                y=type_percentages.index,
                x=type_percentages[True],
                name='Favorite Won',
                marker_color='green',
                orientation='h',
                textposition='none',
                hovertemplate=[
                    f'Favorite Won: {x:,}<br>Percentage: {perc:.2f}%<extra></extra>' 
                    for x, perc in zip(type_counts[True], type_percentages[True])
                ]
            ))

    elif selected_visualization == 'Spread Coverage':
        type_counts = filtered_df.groupby([column, 'favorite_covered']).size().unstack(fill_value=0)
        type_percentages = (type_counts.T / type_counts.sum(axis=1)).T * 100
        
        if False in type_percentages.columns:
            fig.add_trace(go.Bar(
                y=type_percentages.index,
                x=type_percentages[False],
                name='Not Covered',
                marker_color='red',
                orientation='h',
                textposition='none',
                hovertemplate=[
                    f'Not Covered: {x:,}<br>Percentage: {perc:.2f}%<extra></extra>' 
                    for x, perc in zip(type_counts[False], type_percentages[False])
                ]
            ))
        if True in type_percentages.columns:
            fig.add_trace(go.Bar(
                y=type_percentages.index,
                x=type_percentages[True],
                name='Covered',
                marker_color='green',
                orientation='h',
                textposition='none',
                hovertemplate=[
                    f'Covered: {x:,}<br>Percentage: {perc:.2f}%<extra></extra>' 
                    for x, perc in zip(type_counts[True], type_percentages[True])
                ]
            ))

    # Update layout to add y-axis annotations and title
    fig.update_layout(
        title=f'{game_type} {selected_visualization} Distribution by {selected_type}',
        xaxis_title='Percentage',
        yaxis_title=selected_type,
        barmode='stack',
        showlegend=True,
        height=400,
        margin=dict(l=100, r=300, t=100, b=50),
        xaxis=dict(
            tickvals=list(range(0, 101, 10)),
            ticktext=[f'{i}%' for i in range(0, 101, 10)],
            title_font=dict(size=14, family='Arial', color='black', weight='bold')
        ),
        yaxis=dict(
            tickvals=list(type_percentages.index),
            ticktext=[f'{y}' for y in type_percentages.index],
            title_font=dict(size=14, family='Arial', color='black', weight='bold')
        ),
        shapes=[
            dict(
                type='line',
                x0=50,
                x1=50,
                y0=0,
                y1=1,
                xref='x',
                yref='paper',
                line=dict(color='black', width=2)
            )
        ],
        annotations=[
            dict(
                x=1.1,
                y=0.5,
                xref='paper',
                yref='paper',
                text='# of Games',
                showarrow=False,
                font=dict(size=14, color='black', weight='bold'),
                align='left',
                xanchor='left',
                yanchor='middle',
                textangle=-90
            ),
        ],
        legend=dict(
            orientation='h',
            yanchor='bottom',
            xanchor='right',
            x=1.1,
            y=1.05,
        )
    )

    # Add annotations for total number of games next to each y-value, adjacent to the bars
    for i, y_value in enumerate(type_percentages.index):
        total_games = type_counts.sum(axis=1).loc[y_value]
        fig.add_annotation(
            x=1.02,
            y=i,
            xref='paper',
            yref='y',
            text=f'{total_games}',
            showarrow=False,
            font=dict(size=12, color='black'),
            align='left',
            xanchor='left',
            yanchor='middle',
            textangle=0
        )
    
    return fig

if __name__ == '__main__':
    app2.run_server(debug=True, port=8063)